# Mudando o diretório para o meu Google Drive para não precisar ficar baixando os datasets novamente.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pwd

/content


In [3]:
cd "drive/MyDrive/Doutorado/Disciplinas/[2022.1] [UFF] Processamento de Linguagem Natural - Professora: Aline Marins Paes Carvalho/Trabalhos/Trabalho 2 - POS  e Transfer Learning/"

/content/drive/MyDrive/Doutorado/Disciplinas/[2022.1] [UFF] Processamento de Linguagem Natural - Professora: Aline Marins Paes Carvalho/Trabalhos/Trabalho 2 - POS  e Transfer Learning


In [4]:
!pwd

/content/drive/MyDrive/Doutorado/Disciplinas/[2022.1] [UFF] Processamento de Linguagem Natural - Professora: Aline Marins Paes Carvalho/Trabalhos/Trabalho 2 - POS  e Transfer Learning


# Imports

In [5]:
import pandas as pd
from gensim.models import KeyedVectors
import tensorflow as tf
import numpy as np
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader

In [6]:
!pip3 install pyconll
import pyconll # https://github.com/soutsios/pos-tagger-bert/blob/master/pos_tagger_bert.ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Word Embeddings

Baseado em https://colab.research.google.com/drive/1BuLyMlebp43-3KNn-puezjW5S1CGuSI5?usp=sharing

In [7]:
# !wget http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s50.zip

In [8]:
# !unzip download.php?file=embeddings%2Fword2vec%2Fcbow_s50.zip

In [9]:
# word2vec = KeyedVectors.load_word2vec_format('cbow_s50.txt')
# word2vec

In [10]:
# word2vec['menino']

In [11]:
# word2vec.most_similar('homem')

In [12]:
# word2vec.similarity('menino', 'cachorro')

In [13]:
# word2vec.most_similar(positive=['amar', 'odiando'], negative=['odiar'])

# Pre-Train Word Embedding in PyTorch

Baseado em https://androidkt.com/pre-train-word-embedding-in-pytorch/

In [14]:
# !wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

In [15]:
# !unzip glove.6B.zip

In [16]:
# glove = pd.read_csv('glove.6B.100d.txt', sep=" ", quoting=3, header=None, index_col=0)
# glove_embedding = {key: val.values for key, val in glove.T.items()}

In [17]:
# def create_embedding_matrix(word_index,embedding_dict,dimension):
#   embedding_matrix=np.zeros((len(word_index)+1,dimension))
 
#   for word,index in word_index.items():
#     if word in embedding_dict:
#       embedding_matrix[index]=embedding_dict[word]
#   return embedding_matrix
 
# text=["The cat sat on mat","we can play with model"]
 
# tokenizer=tf.keras.preprocessing.text.Tokenizer(split=" ")
# tokenizer.fit_on_texts(text)
 
# text_token=tokenizer.texts_to_sequences(text)
 
# embedding_matrix=create_embedding_matrix(
#     tokenizer.word_index,
#     embedding_dict=glove_embedding,dimension=EMBEDDINGS_SIZE)

In [18]:
# text_token

In [19]:
# tokenizer.word_index

In [20]:
# vocab_size=embedding_matrix.shape[0]
# vector_size=embedding_matrix.shape[1]
 
# embedding=nn.Embedding(num_embeddings=vocab_size,embedding_dim=vector_size)

In [21]:
# embedding.weight=nn.Parameter(torch.tensor(embedding_matrix,dtype=torch.float32))

In [22]:
# embedding(torch.LongTensor([1]))

In [23]:
# embedding.weight.requires_grad=False

In [24]:
# embedding_vec=embedding(torch.LongTensor(text_token))
# print(embedding)
# print(embedding_vec.shape)

In [25]:
# embedding_dim=vector_size
# lstm=nn.LSTM(embedding_dim,128,bidirectional=True,batch_first=True)(embedding_vec)
# lstm

# Nova tentativa de treinar o GRU com embeddings pré-treinados.

In [80]:
EMBEDDINGS_SIZE = 100

In [26]:
'''
{'can': 7,
 'cat': 2,
 'mat': 5,
 'model': 10,
 'on': 4,
 'play': 8,
 'sat': 3,
 'the': 1,
 'we': 6,
 'with': 9}
 '''

"\n{'can': 7,\n 'cat': 2,\n 'mat': 5,\n 'model': 10,\n 'on': 4,\n 'play': 8,\n 'sat': 3,\n 'the': 1,\n 'we': 6,\n 'with': 9}\n "

In [27]:
# word2index = {token: token_index for token_index, token in enumerate(word2vec.index2word)}
# word2index['menino']

In [28]:
# word2index

In [29]:
# glove_embedding

In [30]:
'''
{'the': array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
        -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
         0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
        -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
         0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
        -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
         0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
         0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
        -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
        -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
        -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
        -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
        -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
        -1.2526  ,  0.071624,  0.70565 ,  0.49744 , -0.42063 ,  0.26148 ,
        -1.538   , -0.30223 , -0.073438, -0.28312 ,  0.37104 , -0.25217 ,
         0.016215, -0.017099, -0.38984 ,  0.87424 , -0.72569 , -0.51058 ,
        -0.52028 , -0.1459  ,  0.8278  ,  0.27062 ]),
 ',': array([-0.10767  ,  0.11053  ,  0.59812  , -0.54361  ,  0.67396  ,
 [...]
'''

"\n{'the': array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,\n        -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,\n         0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,\n        -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,\n         0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,\n        -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,\n         0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,\n         0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,\n        -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,\n        -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,\n        -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,\n        -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,\n        -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,\n        -1.2526

In [31]:
# word2vec_embedding = {}

# for index, token in enumerate(word2vec.vocab):
#     if index < 3:
#         print(f'token: {token}')
#         print(f'embeddings: {word2vec[token]}')

#         {'the': array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
#         -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
#          0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
#         -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
#          0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
#         -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
#          0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
#          0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
#         -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
#         -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
#         -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
#         -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
#         -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
#         -1.2526  ,  0.071624,  0.70565 ,  0.49744 , -0.42063 ,  0.26148 ,
#         -1.538   , -0.30223 , -0.073438, -0.28312 ,  0.37104 , -0.25217 ,
#          0.016215, -0.017099, -0.38984 ,  0.87424 , -0.72569 , -0.51058 ,
#         -0.52028 , -0.1459  ,  0.8278  ,  0.27062 ]),
#  ',': array([-0.10767  ,  0.11053  ,  0.59812  , -0.54361  ,  0.67396  ,
#  [...]
#  '''

In [32]:
# def create_embedding_matrix_word2vec(word_index,embedding_dict,dimension):
#   embedding_matrix=np.zeros((len(word_index)+1,dimension))
 
#   for word,index in word_index.items():
#     if word in embedding_dict:
#       embedding_matrix[index]=embedding_dict[word]
#   return embedding_matrix
 
# embedding_matrix_word2vec=create_embedding_matrix(
#     word2index,
#     embedding_dict=glove_embedding,
#     dimension=EMBEDDINGS_SIZE)

In [33]:
# !wget http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s100.zip

In [34]:
# !unzip download.php?file=embeddings%2Fglove%2Fglove_s100.zip

In [81]:
glove_pt = pd.read_csv(
    'glove_s100.txt', 
    sep=" ", 
    quoting=3, 
    header=None, 
    index_col=0,
    skiprows=1)

In [82]:
glove_pt.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,,,,,,,,,,,,,,,,,,,,,
",",-0.392279,-0.383096,-0.163537,-1.615735,-0.500976,-0.007344,0.465154,-0.527951,-0.646091,-0.321491,...,-0.497059,1.318589,-0.233006,-0.156836,0.445024,0.582988,0.177718,0.774904,2.415028,-0.148173
de,-0.059871,-0.266130,-1.288952,-2.352894,0.414237,0.621193,1.073003,-0.548189,-0.643191,0.966722,...,-0.388322,1.054918,-0.412016,0.565986,0.697279,-0.073342,0.275684,0.049941,2.603193,0.559911
.,-0.316228,0.152535,-0.348308,-1.317098,-0.343182,0.486299,-0.361778,-0.758097,-0.200598,0.647857,...,-0.692086,0.909492,-0.121445,-0.189453,0.740305,0.316062,0.507474,0.699510,2.571814,0.632679
a,-1.186349,-0.385310,-0.378241,-2.474666,0.707291,0.086132,0.244584,-0.624107,-0.237829,0.392953,...,-0.316421,1.011388,0.202223,-0.575371,0.026287,-0.275881,0.052405,0.313878,3.199491,0.827020
o,-0.540110,-0.063359,0.194538,-2.323175,-0.100174,1.276602,0.238280,-0.623534,-0.347138,-0.129575,...,0.323425,1.159988,-0.642872,-1.545277,0.151334,-0.785550,0.704339,1.013909,2.478992,-0.193909


In [83]:
glove_pt.tail()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,,,,,,,,,,,,,,,,,,,,,
~$0.0b,0.023192,0.042274,-0.136690,0.096456,-0.024662,0.064788,-0.119517,-0.017335,0.140895,0.030265,...,0.283668,-0.092165,0.006212,0.041488,-0.003957,-0.003753,0.025884,-0.059045,0.213298,-0.000441
~.,0.109924,-0.039958,-0.114303,0.271989,0.022783,0.019153,-0.197778,-0.027550,0.036257,-0.106391,...,0.012212,-0.060298,0.015915,-0.016765,-0.109300,0.064726,-0.048884,0.034652,-0.218065,0.065903
~>,0.091712,-0.137760,-0.148611,0.280811,-0.040886,-0.156137,-0.345717,0.131649,-0.047881,-0.126024,...,0.091572,0.002053,0.144309,-0.234035,-0.003793,0.038706,0.107760,0.125965,-0.256933,0.107929
~~,-0.029480,-0.069960,-0.124492,0.287964,-0.103688,-0.109228,-0.165033,0.106509,-0.014567,-0.223258,...,0.158334,-0.154956,-0.081261,0.057896,-0.140982,0.074473,-0.101146,-0.107334,0.052051,0.020874
<unk>,0.077257,0.064637,-0.029866,0.150767,-0.004830,-0.024283,-0.078053,0.034162,0.049797,-0.111888,...,0.059529,-0.140611,0.036282,0.008047,-0.034854,0.027526,-0.075541,-0.054225,-0.150991,-0.027147


In [84]:
glove_embedding_pt = {key: val.values for key, val in glove_pt.T.items()}

In [39]:
# glove_embedding_pt

## Carregando o dataset do UD

Baseado em https://github.com/soutsios/pos-tagger-bert/blob/master/pos_tagger_bert.ipynb

In [85]:
# trainset
train_X = list()
train_y = list()
data = pyconll.load_from_file( './datasets/pt_bosque-ud-train.conllu' )
train_tagged_sentences=[]
t=0
for sentence in data:
    tagged_sentence=[]
    X_sentence = list()
    y_sentence = list()
    for token in sentence:
        if token.upos and token.form:
            t+=1
            tagged_sentence.append((token.form.lower(), token.upos))

            X_sentence.append( token.form )
            y_sentence.append( token.upos )
    train_X.append( ' '.join(str(t) for t in X_sentence) )
    train_y.append( [str(tag) for tag in y_sentence] )
    train_tagged_sentences.append(tagged_sentence)

In [86]:
# devset
dev_X = list()
dev_y = list()
data = pyconll.load_from_file( './datasets/pt_bosque-ud-dev.conllu' )
dev_tagged_sentences=[]
t=0
for sentence in data:
    tagged_sentence=[]
    X_sentence = list()
    y_sentence = list()
    for token in sentence:
        if token.upos and token.form:
            t+=1
            tagged_sentence.append((token.form.lower(), token.upos))

            X_sentence.append( token.form )
            y_sentence.append( token.upos )
    dev_X.append( ' '.join(str(t) for t in X_sentence) )
    dev_y.append( [str(tag) for tag in y_sentence] )
    dev_tagged_sentences.append(tagged_sentence)

In [87]:
text_pt = train_X

In [88]:
text_pt_dev = dev_X

In [89]:
# text_pt = ["The cat sat on mat","we can play with model"]
# text_pt = ["PT em o governo", "BRASÍLIA Pesquisa Datafolha publicada hoje revela um dado supreendente: recusando uma postura radical, a esmagadora maioria (77%) dos eleitores quer o PT participando do Governo Fernando Henrique Cardoso.", "Tem sentido -- aliás, muitíssimo sentido."]

max_length = 0
for sentence in text_pt:
    tokens = sentence.split()
    len_tokens = len(tokens)
    if len_tokens > max_length:
        max_length = len_tokens

for index, sentence in enumerate(text_pt):
    tokens = sentence.split()
    len_tokens = len(tokens)
    if len_tokens < max_length:
        difference = max_length - len_tokens
        for i in range(0, difference):
            tokens.append( '<unk>' )
    text_pt[index] = ' '.join(tokens)

for index, sentence in enumerate(text_pt):
    if index < 5:
      tokens = sentence.split()
      print(f'tokens: {tokens}')
      print(f'len(tokens): {len(tokens)}')
      print('-----------------------------------')

print()
print( f'len(text_pt): {len(text_pt)}' )

tokens: ['PT', 'em', 'o', 'governo', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', 

In [90]:
max_length = 0
for sentence in text_pt_dev:
    tokens = sentence.split()
    len_tokens = len(tokens)
    if len_tokens > max_length:
        max_length = len_tokens

for index, sentence in enumerate(text_pt_dev):
    tokens = sentence.split()
    len_tokens = len(tokens)
    if len_tokens < max_length:
        difference = max_length - len_tokens
        for i in range(0, difference):
            tokens.append( '<unk>' )
    text_pt_dev[index] = ' '.join(tokens)

for index, sentence in enumerate(text_pt_dev):
    if index < 5:
      tokens = sentence.split()
      print(f'tokens: {tokens}')
      print(f'len(tokens): {len(tokens)}')
      print('-----------------------------------')

print()
print( f'len(text_pt_dev): {len(text_pt_dev)}' )

tokens: ['Pequenos', 'são', 'agentes', 'de', 'as', 'transformações', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<un

In [91]:
def create_embedding_matrix_pt( word_index, embedding_dict, dimension ):
  embedding_matrix = np.zeros( (len(word_index)+1, dimension) )
 
  for word,index in word_index.items():
    if word in embedding_dict:
      embedding_matrix[index] = embedding_dict[word]
  return embedding_matrix

tokenizer_pt = tf.keras.preprocessing.text.Tokenizer(split=" ")
tokenizer_pt.fit_on_texts(text_pt)
 
# # Trying to solve a bug.
# total_sentences = len(text_pt)
# last_sentence = text_pt[total_sentences - 1]
# last_sentence_tokens = last_sentence.split()
# last_sentence_tokens.append( '<unk>' )
# text_pt[total_sentences - 1] = ' '.join( last_sentence_tokens )

text_token_pt = tokenizer_pt.texts_to_sequences(text_pt)
embedding_matrix_pt = create_embedding_matrix_pt(
    tokenizer_pt.word_index,
    embedding_dict = glove_embedding_pt,
    dimension = EMBEDDINGS_SIZE)

Completando o tamanho das sentenças com o id de token = \<unk\>.

In [94]:
# trainset
max_length = 0
for item in text_token_pt:
    len_item = len(item)
    if len_item > max_length:
        max_length = len_item

for index, item in enumerate(text_token_pt):
    len_item = len(item)
    if len_item < max_length:
        difference = max_length - len_item
        for i in range(0, difference):
            item.append( 1 )
    text_token_pt[index] = item

for index, item in enumerate(text_token_pt):
    if index < 5:
        
        print( f'len(item): {len(item)}' )
        print(item)
        print('-------------------------')
        print()

print('-------------------------')
print()        
last_item_index = len(text_token_pt) - 1
last_item = text_token_pt[last_item_index]
print( f'len(last item): {len(last_item)}' )
print( last_item )

len(item): 204
[436, 5, 4, 52, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
-------------------------

len(item): 204
[1681, 533, 2147, 2917, 73, 1513, 14, 1090, 9454, 6145, 16, 2148, 1884, 3, 3524, 608, 3525, 2, 8, 1682, 282, 4, 436, 6146, 2, 4, 52, 249, 386, 639, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [95]:
# devset
max_length = 0
for item in text_token_pt_dev:
    len_item = len(item)
    if len_item > max_length:
        max_length = len_item

for index, item in enumerate(text_token_pt_dev):
    len_item = len(item)
    if len_item < max_length:
        difference = max_length - len_item
        for i in range(0, difference):
            item.append( 1 )
    text_token_pt_dev[index] = item

for index, item in enumerate(text_token_pt_dev):
    if index < 5:
        
        print( f'len(item): {len(item)}' )
        print(item)
        print('-------------------------')
        print()

print('-------------------------')
print()        
last_item_index = len(text_token_pt_dev) - 1
last_item = text_token_pt_dev[last_item_index]
print( f'len(last item): {len(last_item)}' )
print( last_item )

len(item): 135
[1275, 24, 844, 2, 11, 1276, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
-------------------------

len(item): 135
[36, 18, 19, 2315, 610, 6, 4, 2316, 212, 19, 12, 2317, 2, 11, 162, 163, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
-------------------------

len(item): 135
[2318, 2319, 2320, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [96]:
# trainset
embedding_matrix_pt

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.156559,  0.111655, -0.015184, ..., -0.009003, -0.178693,
        -0.02881 ],
       [-0.059871, -0.26613 , -1.288952, ...,  0.049941,  2.603193,
         0.559911],
       ...,
       [-0.521869, -0.323804,  0.429143, ..., -0.355543,  0.04608 ,
         0.165209],
       [ 0.354441, -0.620891, -0.351661, ...,  0.287087, -0.913356,
         0.603573],
       [-0.4722  ,  0.268814, -0.390876, ...,  0.548052, -0.250022,
         0.388714]])

In [97]:
# devset
embedding_matrix_pt_dev

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.156559,  0.111655, -0.015184, ..., -0.009003, -0.178693,
        -0.02881 ],
       [-0.059871, -0.26613 , -1.288952, ...,  0.049941,  2.603193,
         0.559911],
       ...,
       [ 0.018154,  0.148979, -0.004639, ...,  1.02579 , -0.397611,
        -0.097808],
       [ 0.023733, -0.344766, -1.683688, ...,  0.508497,  0.548053,
        -0.61285 ],
       [-0.453163,  0.616099, -1.446902, ...,  0.45914 ,  1.369714,
        -0.409569]])

Vocab size and pos vocab size

In [116]:
vocab_size_pt = embedding_matrix_pt.shape[0]
vector_size_pt = embedding_matrix_pt.shape[1]

Embedding layers

In [117]:
embedding_pt = nn.Embedding(
    num_embeddings = vocab_size_pt,
    embedding_dim = vector_size_pt)

In [118]:
embedding_pt

Embedding(21120, 100)

In [119]:
embedding_pt.weight = nn.Parameter(
    torch.tensor( 
        embedding_matrix_pt,
        dtype = torch.float32))

In [120]:
embedding_pt

Embedding(21120, 100)

In [121]:
embedding_pt( torch.LongTensor([1]) )

tensor([[ 0.1566,  0.1117, -0.0152,  0.2379, -0.0216, -0.0107,  0.1072,  0.0079,
          0.0494, -0.0778, -0.0899,  0.0529, -0.0396,  0.2029, -0.0374,  0.1199,
          0.0809,  0.1342, -0.0090, -0.0834, -0.0824, -0.0700,  0.1080,  0.2166,
          0.1193, -0.0090, -0.1851, -0.1200,  0.0541, -0.1695, -0.0515,  0.0963,
         -0.0076,  0.0464, -0.0442,  0.0612, -0.0656,  0.0922,  0.2037, -0.0509,
          0.0409, -0.0548,  0.1614, -0.2111,  0.0966,  0.0693, -0.0541, -0.0616,
         -0.0084, -0.0246, -0.1267,  0.0437, -0.0291,  0.0008, -0.0207,  0.0924,
          0.0360,  0.0383,  0.1084,  0.0118, -0.0112, -0.0261,  0.1788, -0.0073,
          0.0591, -0.0003, -0.0276,  0.0295, -0.1592,  0.1003,  0.0111,  0.0169,
         -0.0067,  0.1499, -0.2063, -0.1196,  0.0076, -0.0756,  0.1790,  0.1040,
          0.0168,  0.2922,  0.1378,  0.1308, -0.0529,  0.1065,  0.1111,  0.1954,
          0.0441,  0.0470, -0.0312, -0.2474, -0.0717,  0.0511,  0.0974,  0.0909,
          0.0424, -0.0090, -

Make it untrainable by freezing its gradient.

In [51]:
embedding_pt.weight.requires_grad = False

In [52]:
torch.LongTensor(text_token_pt)

tensor([[ 436,    5,    4,  ...,    1,    1,    1],
        [1681,  533, 2147,  ...,    1,    1,    1],
        [  32,  387,  734,  ...,    1,    1,    1],
        ...,
        [4463,   19, 1705,  ...,    1,    1,    1],
        [4463,   18, 5329,  ...,    1,    1,    1],
        [   8, 6144, 3522,  ...,    1,    1,    1]])

In [53]:
embedding_vec_pt = embedding_pt(
    torch.LongTensor(text_token_pt))
print(embedding_pt)
print(embedding_vec_pt.shape)

Embedding(21120, 100)
torch.Size([7018, 204, 100])


In [54]:
# embedding_dim_pt = EMBEDDINGS_SIZE
# lstm_pt = nn.LSTM(
#     embedding_dim_pt,
#     128,
#     bidirectional = True,
#     batch_first = True)(embedding_vec_pt)

In [55]:
# lstm_pt

# Tagger - GRU

In [56]:
class Tagger(nn.Module):
    def __init__(self,
                 input_dim,
                 output_dim,
                 n_layers,
                 embedding_layer, 
                 embedding_dim=64,
                 hidden_dim=64,
                 dropout=0.5,
                 bidirectional=True,
                 pad_idx=0):
        """Initializes the tagger.
        
        Args:
            input_dim: Size of the input vocabulary, projection
            output_dim: Size of the output vocabulary.
            embedding_dim: Dimension of the word embeddings.
            hidden_dim: Number of units in each LSTM hidden layer.
            bidirectional: Whether or not to use a bidirectional rnn.
        """
        super(Tagger, self).__init__()

        # Store parameters
        self.input_dim = input_dim 
        self.output_dim = output_dim
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.bidirectional = bidirectional
          
        # Define layers
        # self.word_embeddings = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)
        self.word_embeddings = embedding_layer
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers = n_layers, 
                          bidirectional=bidirectional,
                          dropout = dropout if n_layers > 1 else 0)
        
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.activation = nn.LogSoftmax(dim=2)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, lengths=None, hidden=None):
        """Computes a forward pass of the language model.
        
        Args:
            x: A LongTensor w/ dimension [seq_len, batch_size].
            lengths: The lengths of the sequences in x.
            hidden: Hidden state to be fed into the lstm.
            
        Returns:
            net: Probability of the next word in the sequence.
            hidden: Hidden state of the lstm.
        """
        seq_len, batch_size = x.size()
        
        # If no hidden state is provided, then default to zeros.
        if hidden is None:
            if self.bidirectional:
                num_directions = 2
            else:
                num_directions = 1
            hidden = Variable(torch.zeros(num_directions, batch_size, self.hidden_dim))
            if torch.cuda.is_available():
                hidden = hidden.cuda()

        # net = self.word_embeddings(x)
        net = self.word_embeddings
        # Pack before feeding into the RNN.
        if lengths is not None:
            lengths = lengths.data.view(-1).tolist()
            net = pack_padded_sequence(net, lengths)
        net, hidden = self.rnn(net, hidden)
        # Unpack after
        if lengths is not None:
            net, _ = pad_packed_sequence(net)
        net = self.fc(net)
        net = self.activation(net)

        return net, hidden

# Training Model

## Vocabulário

In [57]:
tokenizer_pt.document_count

7018

In [58]:
len(train_y)

7018

In [59]:
vocabulario = []
for index, item in enumerate(tokenizer_pt.word_docs):
    # if index < 5:
    #   print( f'item: {item}' )
    #   print( '----------------------------------' )

    if item not in vocabulario:
        vocabulario.append( item )

# print( f'vocabulario[0]: {vocabulario[0]}' )
# print( f'vocabulario[1]: {vocabulario[1]}' )
# print( f'vocabulario[2]: {vocabulario[2]}' )
# print( f'len(vocabulario): {len(vocabulario)}' )

In [60]:
vocabulario_pos = []
for index, item in enumerate(train_y):
    
    # if index < 5:
    #     print( f'item: {item}' )
    #     print( '----------------------------------' )

    for tag in item:
        if tag not in vocabulario_pos:
            vocabulario_pos.append( tag )

# print( f'vocabulario_pos[0]: {vocabulario_pos[0]}' )
# print( f'vocabulario_pos[1]: {vocabulario_pos[1]}' )
# print( f'vocabulario_pos[2]: {vocabulario_pos[2]}' )
# print( f'vocabulario_pos[3]: {vocabulario_pos[3]}' )
# print( f'vocabulario_pos[4]: {vocabulario_pos[4]}' )
# print( f'vocabulario_pos[5]: {vocabulario_pos[5]}' )
# print( f'len(vocabulario_pos): {len(vocabulario_pos)}' )

In [61]:
vocabulario_pos

['PROPN',
 'ADP',
 'DET',
 'NOUN',
 'VERB',
 'ADV',
 'ADJ',
 'PUNCT',
 'NUM',
 'SYM',
 'PRON',
 'AUX',
 'CCONJ',
 'SCONJ',
 'X',
 'INTJ',
 'PART']

In [62]:
vocabulario_pos.append('<unk>')

In [63]:
vocabulario_pos

['PROPN',
 'ADP',
 'DET',
 'NOUN',
 'VERB',
 'ADV',
 'ADJ',
 'PUNCT',
 'NUM',
 'SYM',
 'PRON',
 'AUX',
 'CCONJ',
 'SCONJ',
 'X',
 'INTJ',
 'PART',
 '<unk>']

In [64]:
vocabulario_pos_dict = {}
for index, tag in enumerate(vocabulario_pos):
    vocabulario_pos_dict[tag] = (index + 1)
vocabulario_pos_dict

{'<unk>': 18,
 'ADJ': 7,
 'ADP': 2,
 'ADV': 6,
 'AUX': 12,
 'CCONJ': 13,
 'DET': 3,
 'INTJ': 16,
 'NOUN': 4,
 'NUM': 9,
 'PART': 17,
 'PRON': 11,
 'PROPN': 1,
 'PUNCT': 8,
 'SCONJ': 14,
 'SYM': 10,
 'VERB': 5,
 'X': 15}

## train_dataset and devset

In [65]:
# trainset
len(text_token_pt[0])

204

In [125]:
# devset
len(text_token_pt_dev[0])

135

In [126]:
# trainset
len(train_y[0])

4

In [127]:
# devset
len(dev_y[0])

6

In [128]:
# tags_0 = []
# for tag in train_y[0]:
#     tag_index = vocabulario_pos_dict[ tag ]
#     tags_0.append(tag_index)

# item_dict = (text_token_pt[0], tags_0)
# # item_dict

In [132]:
# trainset
train_dataset = []
for index, item in enumerate(text_token_pt):

    if index < 5: 
        print( item )
        len_item = len(item)
        print( f'len_item: {len_item}' )

    tags = []
    for tag in train_y[ index ]:
        tag_index = vocabulario_pos_dict[ tag ]
        tags.append( tag_index )

    len_item = len(item)
    difference = len_item - len(train_y[ index ])
    for i in range(0, difference):
        tag_index = vocabulario_pos_dict[ '<unk>' ]
        tags.append( tag_index )

    if index < 5: 
        print( f'len(tags): {len(tags)}' )
        print( '-----------------------------------' )

    item_dict = ( text_token_pt[ index ], tags )
    train_dataset.append( item_dict )

print()
print( f'train_dataset[0]: {train_dataset[0]}' )
print( f'train_dataset[1]: {train_dataset[1]}' )
print( f'train_dataset[2]: {train_dataset[2]}' )
print( f'train_dataset[3]: {train_dataset[3]}' )
print( f'train_dataset[4]: {train_dataset[4]}' )

[436, 5, 4, 52, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
len_item: 204
len(tags): 204
-----------------------------------
[1681, 533, 2147, 2917, 73, 1513, 14, 1090, 9454, 6145, 16, 2148, 1884, 3, 3524, 608, 3525, 2, 8, 1682, 282, 4, 436, 6146, 2, 4, 52, 249, 386, 639, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [133]:
# devset
dev_dataset = []
for index, item in enumerate(text_token_pt_dev):

    if index < 5: 
        print( item )
        len_item = len(item)
        print( f'len_item: {len_item}' )

    tags = []
    for tag in dev_y[ index ]:
        tag_index = vocabulario_pos_dict[ tag ]
        tags.append( tag_index )

    len_item = len(item)
    difference = len_item - len(dev_y[ index ])
    for i in range(0, difference):
        tag_index = vocabulario_pos_dict[ '<unk>' ]
        tags.append( tag_index )

    if index < 5: 
        print( f'len(tags): {len(tags)}' )
        print( '-----------------------------------' )

    item_dict = ( text_token_pt_dev[ index ], tags )
    dev_dataset.append( item_dict )

print()
print( f'dev_dataset[0]: {dev_dataset[0]}' )
print( f'dev_dataset[1]: {dev_dataset[1]}' )
print( f'dev_dataset[2]: {dev_dataset[2]}' )
print( f'dev_dataset[3]: {dev_dataset[3]}' )
print( f'dev_dataset[4]: {dev_dataset[4]}' )

[1275, 24, 844, 2, 11, 1276, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
len_item: 135
len(tags): 135
-----------------------------------
[36, 18, 19, 2315, 610, 6, 4, 2316, 212, 19, 12, 2317, 2, 11, 162, 163, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
len_item: 135
len(tags): 135
-----------------------------------
[2318, 2319, 2320, 1, 1, 

In [69]:
''' # Com base no tagger_gru.ipynb

train_dataset[0]
([235, 19, 7, 5, 75], [6, 7, 3, 2, 1])

'''

' # Com base no tagger_gru.ipynb\n\ntrain_dataset[0]\n([235, 19, 7, 5, 75], [6, 7, 3, 2, 1])\n\n'

## Training...

In [134]:
# dev_dataset.token_vocab = vocabulario
# dev_dataset.pos_vocab = vocabulario_pos

# Hyperparameters / constants.
input_vocab_size = tokenizer_pt.document_count
output_vocab_size = len( train_y )
batch_size = 16
epochs = 1
n_layers = 1

# Initialize the model.
model = Tagger(
    input_vocab_size, 
    output_vocab_size, 
    n_layers,
    embedding_layer = embedding_vec_pt)
if torch.cuda.is_available():
    model = model.cuda()

# Loss function weights.
weight = torch.ones(output_vocab_size)
weight[0] = 0
if torch.cuda.is_available():
    weight = weight.cuda()
    
# Initialize loss function and optimizer.
loss_function = torch.nn.NLLLoss(weight)
optimizer = torch.optim.Adam(model.parameters())

# Main training loop.
data_loader = DataLoader(
    train_dataset, 
    batch_size = batch_size, 
    shuffle = True)
dev_loader = DataLoader(
    dev_dataset, 
    batch_size = batch_size, 
    shuffle = True)

losses = []
i = 0
for epoch in range(epochs):
    for inputs, targets, lengths in data_loader:
        optimizer.zero_grad()
        outputs, _ = model(inputs, lengths=lengths)

        outputs = outputs.view(-1, output_vocab_size)
        # print('---------------outputs----------------')
        # print(outputs.shape)
        # print(outputs)
        targets = targets.view(-1)
        # print('---------------targets----------------')
        # print(targets.shape)
        # print(targets)

        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()

        #losses.append(loss.data[0])
        losses.append(loss.item())
        if (i % 10) == 0:
            # Compute dev loss over entire dev set.
            # NOTE: This is expensive. You may want to only use a 
            # subset of the dev set.
            #print('iteration, ', i)
            dev_losses = []
            for inputs, targets, lengths in dev_loader:
                outputs, _ = model(inputs, lengths=lengths)
                outputs = outputs.view(-1, output_vocab_size)
                targets = targets.view(-1)
                loss = loss_function(outputs, targets)
                dev_losses.append(loss.item())
            avg_train_loss = np.mean(losses)
            avg_dev_loss = np.mean(dev_losses)
            losses = []
            #print('here')
            print('Epoch %i Iteration %i - Train Loss: %0.6f - Dev Loss: %0.6f' % (epoch, i, avg_train_loss, avg_dev_loss), end='\n')
            torch.save(model, 'pos_tagger_gru.pt')
        i += 1
        
torch.save(model, 'pos_tagger_gru.final.pt')

ValueError: ignored